# Experimentos en el escenario de cruce

Cargamos librerías.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

from navground import core, sim

plots_dir = os.path.join("plots", "realistic")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

Ejecutamos los experimentos y los guardamos en ficheros .h5

In [ ]:
# Cross
side = 5
num_steps = 300
num_runs = 12
tolerance = 0.5 # Tolerance to target in Cross
behaviour_list = ["ORCA", "HL", "HRVO", "Dummy"]
marker_behaviour = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*"}
color_behaviour = {}
for i, behaviour in enumerate(behaviour_list):
    color_behaviour[behaviour] = mpl.colormaps["Set1"](i / (len(behaviour_list) +1))
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"realistic_{behaviour}.h5")
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: {num_runs}
    record_collisions: true
    record_deadlocks: true
    record_safety_violation: true
    record_efficacy: true
    terminated_when_idle_or_stuck: false
    scenario:
      type: Cross 
      side: {side} 
      tolerance: {tolerance}
      groups:
        - number: 10
          type: human
          radius:
            sampler: uniform
            from: 0.15
            to: 0.3
          color: blue
          kinematics:
            type: Ahead
            max_speed: 1.5
            max_angular_speed: 3.0
          behavior:
            type: HL
            optimal_speed: 
              sampler: uniform
              from: 0.7
              to: 1.2
        - number: 2
          type: wheelchair
          color: red
          radius: 0.4
          kinematics:
            type: 2WDiff
            wheel_axis: 0.6
            max_speed: 1.2
          behavior:
            type: {behaviour}
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

Cargamos los experimentos y los guardamos en un diccionario.

In [ ]:
runs = {}
# Reload simulations
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"realistic_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

Visualizamos alguna ejecución.

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

In [ ]:
notebook_view(width=300)

In [ ]:
rt_sim = RealTimeReplay(run=runs["ORCA"][0], factor=3, web_ui=web_ui)
await rt_sim.run()

Funciones para medir deadlocks, colisiones y eficacia (provenientes de una libreta de Jerome)

In [ ]:
import pandas as pd

def count_deadlocks(deadlock_time, initial_time, final_time):
    is_deadlocked = np.logical_and(deadlock_time > initial_time, deadlock_time < (final_time - 5.0))
    return sum(is_deadlocked)

def extract_data(experiment, initial_step, final_step):
    collisions = []
    deadlocks = []
    efficacy = []
    sms = []
    seeds = []
    for i, run in experiment.runs.items():
        world = run.world
        initial_time, final_time = initial_step*run.time_step, final_step*run.time_step
        deadlocks.append(count_deadlocks(np.array(run.deadlocks), initial_time, final_time))
        collisions.append(np.sum(np.logical_and(
            initial_step < run.collisions[:,0], run.collisions[:,0] < final_step
        )))
        efficacy.append(np.array(run.efficacy[initial_step:final_step]).mean())

    df = pd.DataFrame({
        'deadlocks': deadlocks,
        'collisions': collisions,
        'efficacy': efficacy})
    df['safe'] = (df.collisions == 0).astype(int)
    df['fluid'] = (df.deadlocks == 0).astype(int)
    df['ok'] = ((df.deadlocks == 0) & (df.collisions == 0)).astype(int)
    return df

In [ ]:
steps_list = list(range(40, num_steps, 5))
initial_step = steps_list[0]
final_step = steps_list[-1]
deadlocks = {}
collisions = {}
efficacy = {}
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"realistic_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    df = extract_data(recorded_experiment, initial_step, final_step)
    collisions[behaviour] =  list(df.collisions)
    deadlocks[behaviour] = list(df.deadlocks)
    efficacy[behaviour] = list(df.efficacy)

Visualizamos los rendimientos

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), ncols=3)
for behaviour in behaviour_list:
    ax[0].scatter(deadlocks[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[0].set_xlabel("deadlocks")
    ax[0].set_ylabel("collisions")
    ax[1].scatter(efficacy[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[1].set_xlabel("efficacy")
    ax[1].set_ylabel("collisions")
    ax[2].scatter(efficacy[behaviour], deadlocks[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[2].set_xlabel("efficacy")
    ax[2].set_ylabel("deadlocks")
# end for
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "efficiency_collisions_cross.png")))